In [ ]:
# Setup library
## install -r requirements.txt
import os
import csv
import pickle

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

import tensorflow as tf
tf.random.set_seed(99)

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
### Global variables
IMG_SHAPE = (375, 4)

path_pkl = 'sampled_dataset.pickle'
dataset = pd.read_pickle(path_pkl)

path_index = 'get_index.pickle'
with open(path_index, 'rb') as f:
    idx2lab, lab2cnt = pickle.load(f)

In [ ]:
def pre_reshape(x):
    return np.expand_dims(np.array(x).reshape(IMG_SHAPE)/255, axis=-1)
dataset['x'] = dataset['vector'].apply(pre_reshape)
dataset['y'] = dataset['idx']

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(dataset['x'], dataset['y'],
                                                    test_size=0.2)

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((x_train.values.tolist(), y_train.values.tolist()))
test_dataset = tf.data.Dataset.from_tensor_slices((x_test.values.tolist(), y_test.values.tolist()))

In [ ]:
BATCH_SIZE = 32
train_dataset = train_dataset.batch(BATCH_SIZE)
test_dataset = test_dataset.batch(BATCH_SIZE)

In [ ]:
labset = set()

fig = plt.figure(figsize=(8*2, 4))
fig.set_facecolor('white')
nimg = len(idx2lab.keys())
cnt = 0
labs = sorted(lab2cnt.keys())
for i in range(0, len(x_train)):
    vector = np.array(x_train[x_train.index[i]]).reshape(-1)
    if sum(vector == 0) > 1500-4*25:
        continue
    lab = idx2lab[y_train[y_train.index[i]]]
    if lab != labs[cnt]:
        continue
    ax = fig.add_subplot(1, nimg, cnt+1)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.grid(False)
    ax.imshow(vector[:4*25].reshape((-1, 4)), cmap='gray')
    ax.set_xlabel(lab, color='black', fontsize='large')
    cnt = cnt + 1
    if cnt >= nimg:
        break
fig.savefig('packet_vector.png', bbox_inches='tight')

In [ ]:
# Create Deep and Wide CNN
def create_model(nclass, img_shape=IMG_SHAPE):
    img_input = tf.keras.Input(shape=img_shape+(1, ))
    features1 = tf.keras.layers.Conv2D(32, (1, 1), activation='relu')(img_input)
    features1 = tf.keras.layers.Flatten()(features1)

    features2 = tf.keras.layers.Conv2D(32, (1, 2), activation='relu')(img_input)
    features2 = tf.keras.layers.Flatten()(features2)

    features3 = tf.keras.layers.Conv2D(32, (1, 4), activation='relu')(img_input)
    features3 = tf.keras.layers.Flatten()(features3)

    features4 = tf.keras.layers.Conv2D(32, (2, 2), activation='relu')(img_input)
    features4 = tf.keras.layers.MaxPooling2D((2, 2), strides=(1, 1))(features4)
    
    features5 = tf.keras.layers.Conv2D(32, (2, 2), activation='relu')(features4)

    features4 = tf.keras.layers.Flatten()(features4)
    features5 = tf.keras.layers.Flatten()(features5)

    x = tf.keras.layers.concatenate([features1, features2, features3, features4, features5])

    pred = tf.keras.layers.Dense(nclass)(x)

    model = tf.keras.Model(inputs=[img_input],
                           outputs=[pred])
    return model

In [ ]:
## Log class
### https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/Callback
class CollectBatchStats(tf.keras.callbacks.Callback):
    def __init__(self):
        self.batch_losses = []
        self.batch_val_losses = []
        self.batch_acc = []
        self.batch_val_acc = []
    
    def on_epoch_end(self, epoch, logs=None):
        self.batch_losses.append(logs['loss'])
        self.batch_acc.append(logs['accuracy'])
        self.batch_val_losses.append(logs['val_loss'])
        self.batch_val_acc.append(logs['val_accuracy'])
        self.model.reset_metrics()

In [ ]:
model = create_model(len(idx2lab))
model.summary()

tf.keras.utils.plot_model(model, show_shapes=True)

In [ ]:
# Train build
## Compile model for train
### tf.keras.optimizers.Adam() default: 0.001 
### tf.keras.optimizers.SGD() default: 0.01
base_learning_rate = 0.01 
model.compile(
    optimizer=tf.keras.optimizers.SGD(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'])

In [ ]:
initial_epoch = 10
batch_stats_callback = CollectBatchStats()
earlystop_callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0.001, patience=3)

history = model.fit(train_dataset,
                    epochs=initial_epoch,
                    validation_data=test_dataset,
#                     callbacks=[batch_stats_callback, earlystop_callback])
                    callbacks=[batch_stats_callback])

In [ ]:
# Draw learning curves chart
acc = batch_stats_callback.batch_acc
val_acc = batch_stats_callback.batch_val_acc
loss = batch_stats_callback.batch_losses
val_loss = batch_stats_callback.batch_val_losses

fig2 = plt.figure(figsize=(8, 8))
ax1 = fig2.add_subplot(2, 1, 1)
ax1.plot(acc, label='Training Accuracy')
ax1.plot(val_acc, label='Validation Accuracy')
ax1.legend(loc='lower right')
ax1.set_ylabel('Accuracy')
ax1.set_ylim([0, 1])
ax1.set_title('Training and Validation Accuracy')

ax2 = fig2.add_subplot(2, 1, 2)
ax2.plot(loss, label='Training Loss')
ax2.plot(val_loss, label='Validation Loss')
ax2.legend(loc='upper right')
ax2.set_ylabel('Cross Entropy')
ax2.set_ylim([0, max(ax2.get_ylim())])
ax2.set_title('Training and Validation Loss')
ax2.set_xlabel('epoch')

In [ ]:
with open('centralized_history.pickle', 'wb') as f:
    pickle.dump((acc, val_acc, loss, val_loss), f)
print(max(val_acc))

In [ ]:
model.save_weights('centralized_model/my_model')

In [ ]:
class_names = list()
for i in range(0, len(idx2lab)):
    class_names.append(idx2lab[i])
    
predicted_batch = model.predict(test_dataset)
predicted_id = np.argmax(predicted_batch, axis=-1)

label_id = y_test.values.astype('int64')

con_mat = tf.math.confusion_matrix(label_id, predicted_id)

result_df = pd.DataFrame(con_mat.numpy(), index=class_names, columns=class_names, dtype=int)

print('-- Validation result (Row: Actual Class, Column: Predicted Class) --')
display(result_df)

In [ ]:
true_positive = np.diag(result_df)

true_negative = list()
for i in result_df.index:
    t = result_df.drop(i, axis=1)
    t = t.drop(i, axis=0)
    true_negative.append(np.sum(np.sum(t)))
true_negative = np.asarray(true_negative)

false_positive = np.sum(result_df, axis=1) - true_positive

false_negative = np.sum(result_df, axis=0) - true_positive

print(f'True positive: {true_positive}')
print(f'True negative: {true_negative}')
print(f'False positive: {false_positive}')
print(f'False negative: {false_negative}')

# Draw intermediate image


In [ ]:
outputs = [model.layers[i].output for i in (1, 3, 4, 5, 6)]
intermediate_model = tf.keras.models.Model(inputs=model.input,
                                          outputs=outputs)

In [ ]:
for target in ['youtube', ]:
    ptr = -1
    while True:
        ptr = ptr + 1
        if idx2lab[y_test[y_test.index[ptr]][0]] != target:
            continue
        break
    intermediate_output = intermediate_model.predict(tf.expand_dims(x_test[x_test.index[ptr]], 0))
    for idx in range(0, 5):
        data = intermediate_output[idx]
        fig = plt.figure(figsize=(32, math.ceil(data.shape[-1]/32)*4))
        fig.set_facecolor('white')
        for i in range(0, 32):
            ax = fig.add_subplot(math.ceil(data.shape[-1]/32), 32, i+1)
            ax.imshow(data[0, :10, :, i], cmap='binary')
            ax.set_xticks([])
            ax.set_yticks([])
            ax.grid(False)

# Draw confusion matrix

In [ ]:
classes = list()
for i, l in sorted(idx2lab.items()):
    print(i, l)
    classes.append(l)
print(classes)

In [ ]:
predicted_batch = model.predict(test_dataset)
predicted_id = np.argmax(predicted_batch, axis=-1)

In [ ]:
con_mat = tf.math.confusion_matrix(np.squeeze(y_test.to_list()), predicted_id)
con_mat = con_mat.numpy()

In [ ]:
con_mat_norm = np.around(con_mat.astype('float') / con_mat.sum(axis=1)[:, np.newaxis], decimals=2)
con_mat_df = pd.DataFrame(con_mat_norm, index = classes, columns = classes)

In [ ]:
import plot_helper

In [ ]:
fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot()
im, cbar = plot_helper.heatmap(con_mat_df, cmap=plt.cm.Reds, row_labels=classes, col_labels=classes, ax=ax)
im.set_clim(0, 1)
texts = plot_helper.annotate_heatmap(im)
fig.savefig('confusion_matrix.png', bbox_inches='tight')

# Dynamic

In [ ]:
classes2 = classes
classes = list()
reorder = [7, 0, 14, 8, 11, 2, 15, 10, 4, 13, 5, 12, 1, 3, 9, 6]
for i in reorder:
    classes.append(classes2[i])
print(classes)

In [ ]:
true_id = list()
for p in np.squeeze(y_test.to_list()):
    true_id.append(reorder.index(p))
true_id = np.array(true_id)

In [ ]:
def heatmap(data, row_labels, col_labels, ax=None,
            cbar_kw={}, cbarlabel="", **kwargs):
    """
    Create a heatmap from a numpy array and two lists of labels.

    Parameters
    ----------
    data
        A 2D numpy array of shape (N, M).
    row_labels
        A list or array of length N with the labels for the rows.
    col_labels
        A list or array of length M with the labels for the columns.
    ax
        A `matplotlib.axes.Axes` instance to which the heatmap is plotted.  If
        not provided, use current axes or create a new one.  Optional.
    cbar_kw
        A dictionary with arguments to `matplotlib.Figure.colorbar`.  Optional.
    cbarlabel
        The label for the colorbar.  Optional.
    **kwargs
        All other arguments are forwarded to `imshow`.
    """

    if not ax:
        ax = plt.gca()

    # Plot the heatmap
    im = ax.imshow(data, **kwargs)

    # Create colorbar
    cbar = ax.figure.colorbar(im, ax=ax, **cbar_kw)
    cbar.ax.set_ylabel(cbarlabel, rotation=-90, va="bottom")

    # We want to show all ticks...
    ax.set_xticks(np.arange(data.shape[1]))
    ax.set_yticks(np.arange(data.shape[0]))
    # ... and label them with the respective list entries.
    ax.set_xticklabels(col_labels)
    ax.set_yticklabels(row_labels)

    # Let the horizontal axes labeling appear on top.
    ax.tick_params(top=True, bottom=False,
                   labeltop=True, labelbottom=False)

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=-30, ha="right",
             rotation_mode="anchor")

    # Turn spines off and create white grid.
    for edge, spine in ax.spines.items():
        spine.set_visible(False)

    ax.set_xticks(np.arange(data.shape[1]+1)-.5, minor=True)
    ax.set_yticks(np.arange(data.shape[0]+1)-.5, minor=True)
    ax.grid(which="minor", color="w", linestyle='-', linewidth=3)
    ax.tick_params(which="minor", bottom=False, left=False)

    return im, cbar


def annotate_heatmap(im, data=None, valfmt="{x:.2f}",
                     textcolors=["black", "white"],
                     threshold=None, **textkw):
    """
    A function to annotate a heatmap.

    Parameters
    ----------
    im
        The AxesImage to be labeled.
    data
        Data used to annotate.  If None, the image's data is used.  Optional.
    valfmt
        The format of the annotations inside the heatmap.  This should either
        use the string format method, e.g. "$ {x:.2f}", or be a
        `matplotlib.ticker.Formatter`.  Optional.
    textcolors
        A list or array of two color specifications.  The first is used for
        values below a threshold, the second for those above.  Optional.
    threshold
        Value in data units according to which the colors from textcolors are
        applied.  If None (the default) uses the middle of the colormap as
        separation.  Optional.
    **kwargs
        All other arguments are forwarded to each call to `text` used to create
        the text labels.
    """

    if not isinstance(data, (list, np.ndarray)):
        data = im.get_array()

    # Normalize the threshold to the images color range.
    if threshold is not None:
        threshold = im.norm(threshold)
    else:
        threshold = im.norm(data.max())/2.

    # Set default alignment to center, but allow it to be
    # overwritten by textkw.
    kw = dict(horizontalalignment="center",
              verticalalignment="center")
    kw.update(textkw)

    # Get the formatter in case a string is supplied
    if isinstance(valfmt, str):
        valfmt = matplotlib.ticker.StrMethodFormatter(valfmt)

    # Loop over the data and create a `Text` for each "pixel".
    # Change the text's color depending on the data.
    texts = []
    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            kw.update(color=textcolors[int(im.norm(data[i, j]) > threshold)])
            text = im.axes.text(j, i, valfmt(data[i, j], None), **kw)
            texts.append(text)

    return texts